In [26]:
import pandas as pd
from stop_words import get_stop_words
import nltk
import unicodedata
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [2]:
train = pd.read_csv('manual_sentences.csv')
len(train)

100

In [4]:
train.head()

,text,classification
0,Eu adoro você,pos
1,Eu te adoro,pos
2,Você é uma pessoa maravilhosa,pos
3,Gosto demais de conversar contigo,pos
4,Acho muito divertido ter você por perto,pos


In [5]:
train['classification'] = train['classification'].apply(lambda x: 1 if x == 'pos' else 0)

In [10]:
def remove_accent_from_words(word):
    nfkd = unicodedata.normalize('NFKD', word)
    withouth_accent = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\]', '', withouth_accent)

def normalize(df):
    stemmer = nltk.stem.RSLPStemmer()
    stop_words = get_stop_words('pt')
    for index, row in df.iterrows():
        split = str(row['text']).split()
        split = [word.lower() for word in split]
        split = [stemmer.stem(word) for word in split]
        for word in stop_words:
            if word in split:
                split.remove(word)             
        split = [remove_accent_from_words(word) for word in split]
        df.loc[index, 'text'] = ' '.join(split)
normalize(train)

In [21]:
X = train['text'].tolist()
y = train['classification'].tolist()
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.33, random_state=42)

In [24]:
txt_cl = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])
txt_cl.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [27]:
pred = txt_cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.68      0.93      0.79        14
           1       0.93      0.68      0.79        19

   micro avg       0.79      0.79      0.79        33
   macro avg       0.81      0.81      0.79        33
weighted avg       0.82      0.79      0.79        33

